In [108]:
using Revise

using Kate

### Test Environment

In [2]:
Kate.test()

Testing environment...
`which skewer`
/opt/conda/bin/skewer
`which fastqc`
/opt/conda/bin/fastqc
`which bgzip`
/opt/conda/bin/bgzip
`which tabix`
/opt/conda/bin/tabix
`which minimap2`
/opt/conda/bin/minimap2
`which samtools`
/opt/conda/bin/samtools
`which bcftools`
/opt/conda/bin/bcftools
`which kallisto`
/opt/conda/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/opt/conda/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py


### Set Parameters

In [110]:
# Experiment 
ex = "human_cdna_ashwyn_sharma"

# Fragment length (not needed if paired end)
fr=51

# Standard deviation of read length (not needed if paired end)
sd=0.05

# Reference transcriptome
paf = "../input/human_reference_transcriptome/Homo_sapiens.GRCh38.cdna.all.fa.gz"

pat = "../input/human_reference_transcriptome/enst_gene_name.tsv"

n_jo = 6;

In [111]:
pai = joinpath("../input/") 

pae = joinpath(pai, ex)

pao = joinpath("../output/", ex);

### Check reads

In [112]:
re_ = Kate.find(pae)

Walking sample directory: ../input/human_cdna_ashwyn_sharma

Number of .fastq or .fq files found in directories walked: 0

Number of fastq.gz or fq.gz files found in directories walked: 30

Done at 2021-08-21T23:09:07.758 in 92 milliseconds.



30-element Array{Any,1}:
 "../input/human_cdna_ashwyn_sharma/503DCC-4_R1_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/503DCC-4_R2_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/516BLU-4-resub_R1_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/516BLU-4-resub_R2_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/FX503DCC-3_R1_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/FX503DCC-3_R2_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/FX516BLU-3_R1_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/FX516BLU-3_R2_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/GIST-T1-2_R1_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/GIST-T1-2_R2_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/GIST-T11_R1_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/GIST-T11_R2_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/T1DCC-4_R1_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/T1DCC-4_R2_001.fastq.gz"
 "../input/human_cdna_ashwyn_sharma/T1DCC3_R1_001.fastq.gz"
 "../input/human_cdna_ashwy

In [ ]:
Kate.check(
    re_,
    joinpath(pao, ex, string("check_raw")),
    n_jo,
)

### Count transcript

In [113]:
na_ = ["R1", "read1"]

naf = ""

pap = joinpath(pao, "psuedoalign/")

if !isdir(pap)

    for fq1 in re_

        for na in na_ if occursin(na, fq1)

                naf = na

                nar = replace(naf, "1" => "2")

                fq2 = replace(fq1, naf => nar)

                if !isfile(fq2)

                    fq2 = nothing

                end

                sa = last(splitdir(splitext(split(fq1, naf)[1])[1]))

                println("Working on sample: $sa\n")

                pas = joinpath(pap, sa)

                Kate.count_transcript(
                    paf,
                    pas,
                    n_jo,
                    fq1,
                    fq2,
                    fr,
                    sd,
                )

            end

        end

    end

else

    println("Skipping psuedoalignment because directory already exists: $pap")
    
end

Skipping psuedoalignment because directory already exists: ../output/human_cdna_ashwyn_sharma/psuedoalign/


### Make transcript by sample

In [9]:
using Pkg;

Pkg.add("DataFrames");

   Updating registry at `/opt/julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/Kate.jl/Project.toml`
No Changes to `~/Kate.jl/Manifest.toml`


In [10]:
using Pkg;

Pkg.add("CSV");

  Resolving package versions...
No Changes to `~/Kate.jl/Project.toml`
No Changes to `~/Kate.jl/Manifest.toml`


In [114]:
using CSV

using DataFrames

tp_tr_sa = DataFrame()

for di in readdir(joinpath("../output/", ex, "psuedoalign/"), join = true)
    
    if !occursin("DS_Store", di)
        
        pa = string(joinpath(di, "abundance.tsv"))

        println(pa)
        
        tpm = DataFrame(CSV.File(pa, delim="\t"))[:, [:target_id, :tpm]]
        
        sa = last(splitdir(di))
        
        tpm = rename!(tpm, :tpm => sa)
        
        if isempty(tp_tr_sa)
            
            tp_tr_sa = tpm
            
        else
            
            println(ncol(tp_tr_sa))
            
            id = ncol(tp_tr_sa) + 1
            
            insertcols!(tp_tr_sa, id, sa => tpm[:, sa]) 
        
        end
    
    end

end

tp_tr_sa = rename!(tp_tr_sa, :target_id => :id)

println(first(tp_tr_sa))

CSV.write(joinpath("../output/", ex, "transcript_x_sample.tsv"), tp_tr_sa)

../output/human_cdna_ashwyn_sharma/psuedoalign/.DS_Store
../output/human_cdna_ashwyn_sharma/psuedoalign/503DCC-4_
../output/human_cdna_ashwyn_sharma/psuedoalign/503DCC-4_/abundance.tsv
../output/human_cdna_ashwyn_sharma/psuedoalign/516BLU-4-resub_
../output/human_cdna_ashwyn_sharma/psuedoalign/516BLU-4-resub_/abundance.tsv
2
3
../output/human_cdna_ashwyn_sharma/psuedoalign/FX503DCC-3_
../output/human_cdna_ashwyn_sharma/psuedoalign/FX503DCC-3_/abundance.tsv
3
4
../output/human_cdna_ashwyn_sharma/psuedoalign/FX516BLU-3_
../output/human_cdna_ashwyn_sharma/psuedoalign/FX516BLU-3_/abundance.tsv
4
5
../output/human_cdna_ashwyn_sharma/psuedoalign/GIST-T1-2_
../output/human_cdna_ashwyn_sharma/psuedoalign/GIST-T1-2_/abundance.tsv
5
6
../output/human_cdna_ashwyn_sharma/psuedoalign/GIST-T11_
../output/human_cdna_ashwyn_sharma/psuedoalign/GIST-T11_/abundance.tsv
6
7
../output/human_cdna_ashwyn_sharma/psuedoalign/T1DCC-4_
../output/human_cdna_ashwyn_sharma/psuedoalign/T1DCC-4_/abundance.tsv
7
8
../

"../output/human_cdna_ashwyn_sharma/transcript_x_sample.tsv"

### Make gene by sample

In [115]:
using Statistics

tr_ge = DataFrame(CSV.File(pat, delim="\t"))
    
tr_ge = rename!(tr_ge,  Dict("Transcript stable ID version" => :id, "Gene name" => :gene))
    

# Map transcript to gene name

tp_trge_sa = sort!(innerjoin(tp_tr_sa, tr_ge, on = :id), :gene)

tp_ge__sa = select!(tp_trge_sa, [n for n in names(tp_trge_sa) if n != "id"])


# Save the mean tpm for each gene

gr = groupby(tp_ge__sa, :gene)

sa_ = [n for n in names(tp_trge_sa) if n != "gene"]

tp_ge_sa = DataFrame()

for sa in sa_
    
    ge_sa = combine(gr, sa => mean)
    
    if isempty(tp_ge_sa)
        
        append!(tp_ge_sa, ge_sa)
        
    else
        
        tp_ge_sa = innerjoin(tp_ge_sa, ge_sa, on = :gene)
        
    end
        
end

println(tp_ge_sa[1:5, :])

# Save gene by sample

CSV.write(joinpath("../output/", ex, "gene_x_sample.tsv"), tp_ge_sa)

5×16 DataFrame
 Row │ gene    503DCC-4__mean  516BLU-4-resub__mean  FX503DCC-3__mean  FX516BLU-3__mean  GIST-T1-2__mean  GIST-T11__mean  T1DCC-4__mean  T1DCC3__mean  T1R-DCC-2__mean  T1R-DCC1__mean  T1R-IM-2__mean  T1R-IM1__mean  T1R-SU-2__mean  T1SU-6__mean  T1SU5-resub__mean 
     │ String  Float64         Float64               Float64           Float64           Float64          Float64         Float64        Float64       Float64          Float64         Float64         Float64        Float64         Float64       Float64           
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ A1BG         6.0761                6.7346            6.21009          6.9726            1.46805        1.56077       2.68545         2.329           1.52418         1.35976         1.482

"../output/human_cdna_ashwyn_sharma/gene_x_sample.tsv"